Download PDF

In [1]:
import urllib
import requests

#download pdf from url to local drive
def download_file(download_url):
    response = urllib.request.urlopen(download_url)
    file = open("LordofTheRings.pdf", 'wb')
    file.write(response.read())
    file.close()
    print("Completed")

download_file("https://gosafir.com/mag/wp-content/uploads/2019/12/Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf")

Completed


Extract Pages

In [2]:
import PyPDF2
from cleantext import clean
import re

def read_pdf(pdf_location):
    # creating a pdf reader object
    reader = PyPDF2.PdfReader(pdf_location)

    # print the number of pages in pdf file
    print(f"Total number of pages: {len(reader.pages)}")

    pages = []
    for i in range(len(reader.pages)):
        page = reader.pages[i].extract_text()
        if page != '':
            #replace two or more new lines with just one
            page = re.sub("\n{2,}","\n", page)
            #replace two or more spaces with just one
            page = re.sub("\{2,}","\n", page)
            #clean ascii text and remove any special characters
            page = clean(page, fix_unicode=True, to_ascii=True, lower=False, no_line_breaks=False,
                         no_urls=False, no_emails=False, no_phone_numbers=False, no_numbers=False, 
                         no_digits=False, no_currency_symbols=False, no_punct=False, lang="en")
            #append page and page number to a list of tuples
            pages.append((i, page))
    return pages

In [3]:
pages = read_pdf('LordofTheRings.pdf')

Total number of pages: 1210


Create Whoosh Index

In [1]:
import os
from whoosh.fields import Schema, TEXT, ID
from whoosh.index import create_in, open_dir
from whoosh.qparser import QueryParser

In [42]:
def add_pages_to_index(index_location):
    schema = Schema(path=ID(stored=True), content=TEXT(stored=True))
    
    #create new or load whoosh index
    if not os.path.exists(index_location):
        print("Creating new index.")
        os.mkdir(index_location)
        ix = create_in(index_location, schema)
    else:
        print("Appending to existing index")
        ix = open_dir(index_location)
        
    #create a new writer object
    writer = ix.writer()
    
    #loop through pages and add documents to index
    for page_num, page in pages:
        writer.add_document(path=str(page_num), content=page)
    writer.commit()

In [ ]:
add_pages_to_index('LordOfTheRingsWhooshIndex')

Query Whoosh Index

In [2]:
def query_index(index_location, user_query):
    ix = open_dir(index_location)
    
    with ix.searcher() as searcher:
        #load input query to parser
        query = QueryParser("content", ix.schema).parse(user_query)
        print(f"Query: {query}")
        #search index for query
        results = searcher.search(query, terms=True, limit=10)
        
        results_list = []
        #iterate through response
        for res in results:
            results_list.append((res['path'], res.score, res['content']))
    return query, results_list

In [3]:
query, results = query_index('LordOfTheRingsWhooshIndex',
                             '"one ring to rule them all" "in darkness bind them"')

Query: (content:"one ring rule them all" AND content:"darkness bind them")


In [4]:
for hit in results:
    print(f"\nPage: {hit[0]}")
    print(f"Content:\n{hit[2][:1000]}")
    print(f"-"*56 + "\n")


Page: 73
Content:
50 TH E L ORD OF THE RI NGS
No apparent change came over the ring. After a while Gandalf got
up, closed the shutters outside the window, and drew the curtains.
The room became dark and silent, though the clack of Sam's shears,
now nearer to the windows, could still be heard faintly from the
garden. For a moment the wizard stood looking at the fire; then he
stooped and removed the ring to the hearth with the tongs, and at
once picked it up. Frodo gasped.
'It is quite cool,' said Gandalf. 'Take it!' Frodo received it on his
shrinking palm: it seemed to have become thicker and heavier than
ever.
'Hold it up!' said Gandalf. 'And look closely!'
As Frodo did so, he now saw fine lines, finer than the finest pen-
strokes, running along the ring, outside and inside: lines of fire that
seemed to form the letters of a flowing script. They shone piercingly
bright, and yet remote, as if out of a great depth.
'I cannot read the fiery letters,' said Frodo in a quavering voice.
'No,